In [4]:
import sys
import numpy as np
import pandas as pd
# from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from PIL import Image

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from tqdm import tqdm

from sklearn.utils import shuffle

import numpy as np
import shapely.wkt
import shapely
from shapely.geometry import Polygon

In [5]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [6]:

def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [7]:
# https://medium.com/@prabhnoor0212/siamese-network-keras-31a3a8f37d04
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(16, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer='he_normal', kernel_regularizer=l2(2e-3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (7,7), activation='relu',
                     kernel_initializer='he_normal',
                     bias_initializer='he_normal', kernel_regularizer=l2(2e-3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (4,4), activation='relu', kernel_initializer='he_normal',
                     bias_initializer='he_normal', kernel_regularizer=l2(2e-3)))
    model.add(MaxPooling2D())
#     model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer='he_normal',
#                      bias_initializer='he_normal', kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer='he_normal',bias_initializer='he_normal'))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer='he_normal')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [8]:
model = get_siamese_model((224, 224, 3))

optimizer = Adam(lr = 0.00001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         138741040   input_1[0][0]                    
                                                                 input_2[0][0]                    
________________________________________________________________________________________________

In [9]:
with open('/home/abhirag/experiments_on_portugal/training_data.pkl', 'rb') as fp:
    training_data = pickle.load(fp)

In [10]:
count_no_damage = 0
count_of_destroyed = 0 
data_for_siamese_training = []
for img_name in list(training_data.keys()):
    for uid in list(training_data[img_name].keys()):
        if training_data[img_name][uid]['label'] != 'un-classified':
            if training_data[img_name][uid]['label'] == 'no-damage' and count_no_damage < 250:
                
#                 print(img_name)
#                 print(uid)
                
                
                pre_img = np.array(Image.fromarray(training_data[img_name][uid]['pre']).resize((224, 224))).reshape(224, 224, 3)
                post_img = np.array(Image.fromarray(training_data[img_name][uid]['post']).resize((224, 224))).reshape(224, 224, 3)
                count_no_damage += 1
                data_for_siamese_training.append(([pre_img, post_img], [0], [training_data[img_name][uid]['label']]))
                #model.fit([pre_img, post_img], [0])
            elif training_data[img_name][uid]['label'] != 'no-damage':
                if training_data[img_name][uid]['label'] == 'destroyed' and count_of_destroyed < 250:
                    
                    pre_img = np.array(Image.fromarray(training_data[img_name][uid]['pre']).resize((224, 224))).reshape(224, 224, 3)
                    post_img = np.array(Image.fromarray(training_data[img_name][uid]['post']).resize((224, 224))).reshape(224, 224, 3)
                    data_for_siamese_training.append(([pre_img, post_img], [1],[training_data[img_name][uid]['label']]))
                    count_of_destroyed += 1
                elif training_data[img_name][uid]['label'] != 'destroyed' :
                    pre_img = np.array(Image.fromarray(training_data[img_name][uid]['pre']).resize((224, 224))).reshape(224, 224, 3)
                    post_img = np.array(Image.fromarray(training_data[img_name][uid]['post']).resize((224, 224))).reshape(224, 224, 3)
                    data_for_siamese_training.append(([pre_img, post_img], [1],[training_data[img_name][uid]['label']]))

                    #model.fit([pre_img, post_img], [1])
#         break
#     break

In [11]:
print(len(data_for_siamese_training))


875


In [9]:
damage = [d[2][0] for d in data_for_siamese_training]
#print(damage)
for i in set(damage):
    print('counts of ', i, ' = ', damage.count(i))

counts of  no-damage  =  250
counts of  destroyed  =  250
counts of  major-damage  =  231
counts of  minor-damage  =  144


In [10]:
import random
from tqdm import tqdm
batch_size = 32
epochs = 50 
no_of_batches = int(len(data_for_siamese_training) / batch_size)

def get_batch_data(data_for_siamese_training, index):
    random.shuffle(data_for_siamese_training)
    inp = []
    target = []
    pre_data = []
    post_data = []
    for i in range(index* batch_size, (index+1) * batch_size):
        pre_data.append(np.array(data_for_siamese_training[i][0][0]))
        post_data.append(np.array(data_for_siamese_training[i][0][1]))

        #inp.append([np.array(data_for_siamese_training[i][0][0]), np.array(data_for_siamese_training[i][0][1])])
        #inp.append(data_for_siamese_training[i][0])
        target.append(data_for_siamese_training[i][1][0])

    inp.append(pre_data) 
    inp.append(post_data)
    
    return inp,target

    
for i in range(epochs):
    total_loss = 0
    for j in range(no_of_batches) :
        inp, target = get_batch_data(data_for_siamese_training, j)
        loss = model.train_on_batch(inp,target)
        preds = model.predict_on_batch(inp)
        #print(preds)
        #preds = [1 if pred > 0.5 else 0 for pred in preds]
        #print(preds)
        
        total_loss += loss
        
    print("for epoch : ", i, " loss is : ", total_loss/no_of_batches)



for epoch :  0  loss is :  9.012404335869682
for epoch :  1  loss is :  8.658326360914442
for epoch :  2  loss is :  8.437366061740452
for epoch :  3  loss is :  8.161467693470142
for epoch :  4  loss is :  7.934523140942609
for epoch :  5  loss is :  7.688010798560248
for epoch :  6  loss is :  7.47329655400029
for epoch :  7  loss is :  7.242268350389269
for epoch :  8  loss is :  7.023373462535717
for epoch :  9  loss is :  6.8199291759067115
for epoch :  10  loss is :  6.656363787474455
for epoch :  11  loss is :  6.462194425088388
for epoch :  12  loss is :  6.2926425227412475
for epoch :  13  loss is :  6.129975831067121
for epoch :  14  loss is :  5.987438978972258
for epoch :  15  loss is :  5.856455273098415
for epoch :  16  loss is :  5.727321783701579
for epoch :  17  loss is :  5.613326302281132
for epoch :  18  loss is :  5.496279257315177
for epoch :  19  loss is :  5.367311742570665
for epoch :  20  loss is :  5.265688648930302
for epoch :  21  loss is :  5.181750915668

In [11]:
preds_model = []
lables = []
damage_lables = []
for i in range(len(data_for_siamese_training)):
    aa = []
    aa.append(np.array(data_for_siamese_training[i][0][0]).reshape(1, 224, 224, 3))
    aa.append(np.array(data_for_siamese_training[i][0][1]).reshape(1, 224, 224, 3))

    preds_model.append(model.predict(aa))
    lables.append(data_for_siamese_training[i][1][0])
    damage_lables.append(data_for_siamese_training[i][2][0])
    

In [12]:
preds = [1 if pred > 0.5 else 0 for pred in preds_model]
correct_count = 0
for i in range(len(preds)):
    if preds[i] == lables[i]:
        correct_count += 1
        
print("siamese damage Vs no damage accuracy : ", correct_count/ len(preds) *100)

siamese damage Vs no damage accuracy :  91.77142857142857


In [13]:
preds_model

[array([[0.96743697]], dtype=float32),
 array([[0.91662467]], dtype=float32),
 array([[0.9409592]], dtype=float32),
 array([[0.9335786]], dtype=float32),
 array([[0.9937995]], dtype=float32),
 array([[0.3483474]], dtype=float32),
 array([[0.99768794]], dtype=float32),
 array([[0.7624104]], dtype=float32),
 array([[0.934306]], dtype=float32),
 array([[0.88608646]], dtype=float32),
 array([[0.24677666]], dtype=float32),
 array([[0.9806126]], dtype=float32),
 array([[0.7504544]], dtype=float32),
 array([[0.98084027]], dtype=float32),
 array([[0.63775146]], dtype=float32),
 array([[0.20708853]], dtype=float32),
 array([[0.27294463]], dtype=float32),
 array([[0.97527516]], dtype=float32),
 array([[0.9709145]], dtype=float32),
 array([[0.8227908]], dtype=float32),
 array([[0.950511]], dtype=float32),
 array([[0.93123806]], dtype=float32),
 array([[0.9050821]], dtype=float32),
 array([[0.8290488]], dtype=float32),
 array([[0.5977251]], dtype=float32),
 array([[0.21898054]], dtype=float32),
 a

In [14]:
pred_4_class = []
for pred in preds_model :
    if pred < 0.4 :
        pred_4_class.append('0')
    elif pred < 0.6 :
        pred_4_class.append('1')
    elif pred < 0.8 :
        pred_4_class.append('2')
    else :
        pred_4_class.append('3')

damage_mappings = {'no-damage' : '0', 'minor-damage' : '1', 'major-damage' : '2', 'destroyed': '3'}
damage_gt = [damage_mappings.get(item) for item in damage_lables]
print(len(damage_gt))
print(len(pred_4_class))
correct_count = 0
for i in range(len(pred_4_class)):
    if pred_4_class[i] == damage_gt[i]:
        correct_count += 1
        
print("accuracy : ", correct_count/ len(pred_4_class) *100)

875
875
accuracy :  52.68571428571428


In [15]:
preds

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_pred = pred_4_class, y_true = damage_gt))


              precision    recall  f1-score   support

           0       0.90      0.78      0.84       250
           1       0.22      0.09      0.13       144
           2       0.35      0.16      0.22       231
           3       0.44      0.86      0.58       250

    accuracy                           0.53       875
   macro avg       0.48      0.47      0.44       875
weighted avg       0.51      0.53      0.48       875



In [17]:
model.save("siamese_classification.h5")

In [12]:
from keras.models import load_model

model = load_model('siamese_classification.h5')



In [13]:


import json, os, random
from PIL import Image


def process_img(img_array, polygon_pts, scale_pct):
    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
#     xmin = max(int(xmin - (xdiff * scale_pct)), 0)
#     xmax = min(int(xmax + (xdiff * scale_pct)), width)
#     ymin = max(int(ymin - (ydiff * scale_pct)), 0)
#     ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]

base = '/home/abhirag/ONLY_PORTUGAL/validation/'

images_list = [base + i for i in os.listdir(base)]

random.shuffle(images_list)

predicted_labels = []
actual_labels = []
################################################################################################



for i in images_list:
    
#     i = '/home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001119_post_disaster.png'
    
    if i.split('_')[3] == 'post' and i.split('.')[-1] == 'png':

        
        
        print('PROCESSING +++++++ ', i)
        post_image_to_process = i
        pre_image_to_process = i.replace('_post_', '_pre_')

        pre_img_arr = np.array(Image.open(pre_image_to_process))
        post_img_arr = np.array(Image.open(post_image_to_process))

        # PROCESS THE ABOVE IMAGE USING SPACENET INFERENCE AND GET OUTPUT JSON

        os.system('python3 /home/abhirag/experiments_on_portugal/spacenet/src/models/inference.py --input ' + i + ' --weights /home/abhirag/experiments_on_portugal/spacenet/models/logs/model_iter_711 --mean /home/abhirag/experiments_on_portugal/spacenet_gt/dataSet/mean.npy --output /home/abhirag/experiments_on_portugal/output_json.json')


        data = json.load(open('/home/abhirag/experiments_on_portugal/output_json.json'))



        # POLYGON IS IN HERE == data['features']['xy'][0]['wkt']

        # OUTPUT OF ABOVE COMMAND IS 'POLYGON ((645.0 944.0,645.0 945.0,643.0 947.0,645.0 947.0,645.0 944.0))'

        # polygon_pts IS OUTPUT POLYGON FROM SEGMENTATION MODEl

        # WILL HAVE TO ITERATE OVER ALL POLYGONS IN TRAINING JSON TO CALCULATE MAX IOU OVER ALL POLYGONS

        original_json = json.load(open(post_image_to_process.replace('png', 'json')))

        # THIS DICTIONARY IS OF THE FORM {'uid' : {'IOU' : some percentage, 'damage' : level_of_damage}}

        max_ious = {}

        
        
        
        
        
        for x in range(len(data['features']['xy'])):
            max_ious[data['features']['xy'][x]['properties']['uid']] = {'max' : 0, 'damage' : None}
            polygon_geom = shapely.wkt.loads(data['features']['xy'][x]['wkt'])

            max = 0
            d = None

            for i in range(len(original_json['features']['xy'])):
                polygon_original = shapely.wkt.loads(original_json['features']['xy'][i]['wkt'])
                try:
                    if polygon_geom.intersection(polygon_original):
                        iou = polygon_geom.intersection(polygon_original).area / polygon_geom.union(polygon_original).area
                        if iou > max:
                            max = iou
                            d = original_json['features']['xy'][i]['properties']['subtype']
                        # print(iou)
                except:
                    pass

            if max == 0:
                del max_ious[data['features']['xy'][x]['properties']['uid']]
            else:
                max_ious[data['features']['xy'][x]['properties']['uid']]['max'] = max
                max_ious[data['features']['xy'][x]['properties']['uid']]['damage'] = d

#         print(max_ious)
        
        list_of_polygons = list(max_ious.keys())


        for i in range(len(data['features']['xy'])):
            if data['features']['xy'][i]['properties']['uid'] in list_of_polygons:
                
                actual_labels.append(max_ious[data['features']['xy'][i]['properties']['uid']]['damage'])

                poly = np.array(list(shapely.wkt.loads(data['features']['xy'][i]['wkt']).exterior.coords))

                pre_x = process_img(pre_img_arr, poly, 1.0)
                post_x = process_img(post_img_arr, poly, 1.0)

        #         print('pre = ', pre_x.shape)
        #         print('post = ', post_x.shape)

                pre = np.array(Image.fromarray(pre_x, 'RGB').resize((224, 224))).reshape(1, 224, 224, 3)
                post = np.array(Image.fromarray(post_x, 'RGB').resize((224, 224))).reshape(1, 224, 224, 3)
                
                predicted_labels.append(model.predict([pre, post]))
                
            
#     break

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001626_post_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 131 812 at 131 812
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 131 812 at 131 812
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 131 812 at 131 812


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001051_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001104_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001121_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000583_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001321_post_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 198 596 at 198 596


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000612_post_disaster.png


...

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000683_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000933_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000397_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000410_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000140_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000557_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001119_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000695_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001482_post_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 397 817 at 397 817
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 671 744 at 671 744


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000663_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000119_post_disaster.png


TopologyException: Input geom 0 is invalid: Self-intersection at or near point 871 700 at 871 700


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000686_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001020_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000526_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001258_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001220_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001180_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000120_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001868_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000204_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

In [14]:
predicted_labels



[array([[0.22219007]], dtype=float32),
 array([[0.2479168]], dtype=float32),
 array([[0.26922423]], dtype=float32),
 array([[0.33375576]], dtype=float32),
 array([[0.21271667]], dtype=float32),
 array([[0.24157263]], dtype=float32),
 array([[0.2260262]], dtype=float32),
 array([[0.24154387]], dtype=float32),
 array([[0.2002147]], dtype=float32),
 array([[0.22671439]], dtype=float32),
 array([[0.2282411]], dtype=float32),
 array([[0.20288533]], dtype=float32),
 array([[0.19421248]], dtype=float32),
 array([[0.21124016]], dtype=float32),
 array([[0.21683383]], dtype=float32),
 array([[0.71866214]], dtype=float32),
 array([[0.20201503]], dtype=float32),
 array([[0.29308653]], dtype=float32),
 array([[0.2297414]], dtype=float32),
 array([[0.22445263]], dtype=float32),
 array([[0.18724991]], dtype=float32),
 array([[0.2574888]], dtype=float32),
 array([[0.8506608]], dtype=float32),
 array([[0.5068562]], dtype=float32),
 array([[0.23221181]], dtype=float32),
 array([[0.20508876]], dtype=floa

In [15]:
pred_4_class = []
for pred in predicted_labels :
    if pred < 0.65 :
        pred_4_class.append('no-damage')
    elif pred < 0.75 :
        pred_4_class.append('minor-damage')
    elif pred < 0.85 :
        pred_4_class.append('major-damage')
    else :
        pred_4_class.append('destroyed')

In [16]:
pred_4_class

['no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'minor-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'destroyed',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'destroyed',
 'no-damage',
 'no-damage',
 'no-damage',
 'major-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'destroyed',
 'no-damage',
 'no-damage',
 'no-damage',
 'minor-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'minor-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-dam

In [17]:
actual_labels

['no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'major-damage',
 'major-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'un-classified',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damage',
 'no-damag

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_true = actual_labels, y_pred = pred_4_class))

               precision    recall  f1-score   support

    destroyed       0.00      0.00      0.00         4
 major-damage       0.00      0.00      0.00        16
 minor-damage       0.00      0.00      0.00         3
    no-damage       0.98      0.97      0.97      2542
un-classified       0.00      0.00      0.00        39

     accuracy                           0.94      2604
    macro avg       0.20      0.19      0.19      2604
 weighted avg       0.95      0.94      0.95      2604



/home/abhirag/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
for i in set(pred_4_class):
    print(i, '====', pred_4_class.count(i))

minor-damage ==== 27
no-damage ==== 2515
major-damage ==== 22
destroyed ==== 40


In [20]:
for i in set(actual_labels):
    print(i, '====', actual_labels.count(i))

no-damage ==== 2542
un-classified ==== 39
destroyed ==== 4
major-damage ==== 16
minor-damage ==== 3


In [9]:
data['features']['xy'][0]

{'properties': {'feature_type': 'building',
  'uid': '92abc3b6-4f0d-4520-b6bf-5bf0ae7566ec'},
 'wkt': 'POLYGON ((794.0 469.0,794.0 473.0,795.0 474.0,796.0 474.0,797.0 475.0,803.0 475.0,804.0 474.0,805.0 474.0,805.0 472.0,806.0 471.0,806.0 470.0,805.0 470.0,804.0 469.0,794.0 469.0))'}

In [10]:
max_ious

{'92abc3b6-4f0d-4520-b6bf-5bf0ae7566ec': {'max': 0.4330021367971573,
  'damage': 'no-damage'},
 '2476b5dc-cdee-4335-90f5-ff8790a53f76': {'max': 0.38026472801054817,
  'damage': 'no-damage'},
 '781d4fac-bf5f-4c69-891d-e18aa507a231': {'max': 0.5935944284058069,
  'damage': 'no-damage'},
 '6df9c0a4-acc4-412e-a79a-f1ccbc81787d': {'max': 0.02505655477806296,
  'damage': 'no-damage'},
 '8c8f606c-73fd-4123-9a5f-857427d6ed41': {'max': 0.6518341275948285,
  'damage': 'no-damage'},
 '67630eec-55b2-4d9c-81f0-2cf675eff1cd': {'max': 0.01366721169712525,
  'damage': 'no-damage'},
 'eb3dd7c1-a294-44c9-890b-dfcead2c1bdc': {'max': 0.006140686616366643,
  'damage': 'no-damage'}}